In [ ]:
'''
char_ngram_vocab_size : len(ngrams_dict) +1 => 글자가 몇 개 있는지
word_ngram_vocab_size : len(words_dict) + 1 => 단어가 몇 개 있는지

chars_dict = ngrams_dict 로 두었다. 왜냐하면, 지금 ngram 크기를 1로 두어서 그럼. 

char_vocab_size = len(chars_dict)+1 => 따라서 chars_dict도 ngram으로 한거랑 같게 되는 것임. 

'''
class TextCNN(object): 
    def __init__(self, char_ngram_vocab_size, word_ngram_vocab_size, char_vocab_size, \
        word_seq_len, char_seq_len, embedding_size, l2_reg_lambda=0, \
        filter_sizes=[3,4,5,6], mode=0): 
        if mode == 4 or mode == 5: 
            # 이 경우에는 char-level로 embedding 시키는것도 있어야한다. 
            # 이 때 char는 embedding된 vector를 이용하기 때문에 size가 다음과 같다. 
            self.input_x_char = tf.placeholder(tf.int32, [None, None, None], name="input_x_char")
            self.input_x_char_pad_idx = tf.placeholder(tf.float32, [None, None, None, embedding_size], name="input_x_char_pad_idx")
        if mode == 4 or mode == 5 or mode == 2 or mode == 3: 
            self.input_x_word = tf.placeholder(tf.int32, [None, None], name="input_x_word")
        if mode == 1 or mode == 3 or mode == 5: 
            self.input_x_char_seq = tf.placeholder(tf.int32, [None, None], name="input_x_char_seq") 

        # y는 label이니깐, None 에다가 마지막 차원은 2이다.     
        self.input_y = tf.placeholder(tf.float32, [None, 2], name="input_y") 
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob") 

        l2_loss = tf.constant(0.0) 
        with tf.name_scope("embedding"): 
            
            # mode가 4,5 일 때는 word를 다시 character level로 나누는거 같다. 그래서 unseen word 에 대한 pattern을 학습시키는 것 같다. 
            # 그래서 word를 다시, ngram으로 나눠서 하나의 word를 표현하는 ngram단위로 다시 표현하려고 character-level 이라는게 붙는것같다. 
            if mode == 4 or mode == 5: 
                # char-level일 때, character를 나타내는 embedding vector이다. 
                self.char_w = tf.Variable(tf.random_uniform([char_ngram_vocab_size, embedding_size], -1.0, 1.0), name="char_emb_w") 
            if mode == 2 or mode == 3 or mode == 4 or mode == 5: 
                self.word_w = tf.Variable(tf.random_uniform([word_ngram_vocab_size, embedding_size], -1.0, 1.0), name="word_emb_w")
            if mode == 1 or mode == 3 or mode == 5: 
                
                # 딱 글자 단위로만 잘랐을 때, 그 개개의 character 마다의 embedding 값
                
                self.char_seq_w = tf.Variable(tf.random_uniform([char_vocab_size, embedding_size], -1.0, 1.0), name="char_seq_emb_w")        
     
            if mode == 4 or mode == 5: 
            
                # input으로 들어온 char를 w즉 임베딩된 table에서 맞는걸 찾아서 주는듯? 
                # W가 각 char 별로 embedding 값이 들어있다. 이 때, 내부적으로 어떤 char가 어떤 embedding 값에 매칭되는지를 idx를 내부적으로 정한다. 
                # 그 다음에 input이 들어오면 그것에 해당하는 idx를 구해서, idx를 중심으로 embedding table에 접근해서 값을 가져오는 듯 하다. 
                self.embedded_x_char = tf.nn.embedding_lookup(self.char_w, self.input_x_char)
                
                # 왜 input_x_char_pad_idx랑 곱하지? 
                self.embedded_x_char = tf.multiply(self.embedded_x_char, self.input_x_char_pad_idx)
            
            if mode == 2 or mode == 3 or mode == 4 or mode == 5: 
                self.embedded_x_word = tf.nn.embedding_lookup(self.word_w, self.input_x_word) 
            if mode == 1 or mode == 3 or mode == 5: 
                self.embedded_x_char_seq = tf.nn.embedding_lookup(self.char_seq_w, self.input_x_char_seq)  

            if mode == 4 or mode == 5: 
                self.sum_ngram_x_char = tf.reduce_sum(self.embedded_x_char, 2)         
                self.sum_ngram_x = tf.add(self.sum_ngram_x_char, self.embedded_x_word)

            if mode == 4 or mode == 5: 
                self.sum_ngram_x_expanded = tf.expand_dims(self.sum_ngram_x, -1) 
            if mode == 2 or mode == 3: 
                self.sum_ngram_x_expanded = tf.expand_dims(self.embedded_x_word, -1) 
            if mode == 1 or mode == 3 or mode == 5: 
                self.char_x_expanded = tf.expand_dims(self.embedded_x_char_seq, -1) 
        
        ########################### WORD CONVOLUTION LAYER ################################
        if mode == 2 or mode == 3 or mode == 4 or mode == 5: 
            pooled_x = []

            for i, filter_size in enumerate(filter_sizes):
                with tf.name_scope("conv_maxpool_%s" % filter_size): 
                    filter_shape = [filter_size, embedding_size, 1, 256]
                    b = tf.Variable(tf.constant(0.1, shape=[256]), name="b") 
                    w = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="w") 
                    conv = tf.nn.conv2d(
                        self.sum_ngram_x_expanded,  # weight를 적용할, 주어진 것. 
                        w,
                        strides = [1,1,1,1],
                        padding = "VALID",
                        name="conv")
                    h = tf.nn.relu(tf.nn.bias_add(conv,b), name="relu") 
                    pooled = tf.nn.max_pool(
                        h,
                        ksize=[1, word_seq_len - filter_size + 1, 1, 1],
                        strides=[1,1,1,1],
                        padding="VALID",
                        name="pool") 
                    pooled_x.append(pooled) 
        
            num_filters_total = 256 * len(filter_sizes) 
            self.h_pool = tf.concat(pooled_x, 3)
            self.x_flat = tf.reshape(self.h_pool, [-1, num_filters_total], name="pooled_x")  
            self.h_drop = tf.nn.dropout(self.x_flat, self.dropout_keep_prob, name="dropout_x") 

        ########################### CHAR CONVOLUTION LAYER ###########################
        if mode == 1 or mode == 3 or mode == 5: 
            pooled_char_x = []
            for i, filter_size in enumerate(filter_sizes):
                with tf.name_scope("char_conv_maxpool_%s" % filter_size):
                    filter_shape = [filter_size, embedding_size, 1, 256]
                    b = tf.Variable(tf.constant(0.1, shape=[256]), name="b")
                    w = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="w")
                    conv = tf.nn.conv2d(
                        self.char_x_expanded,
                        w,
                        strides=[1,1,1,1],
                        padding="VALID",
                        name="conv")
                    h = tf.nn.relu(tf.nn.bias_add(conv,b), name="relu")
                    pooled = tf.nn.max_pool(
                        h,
                        ksize=[1, char_seq_len - filter_size + 1, 1, 1],
                        strides=[1,1,1,1],
                        padding="VALID",
                        name="pool")
                    pooled_char_x.append(pooled) 
            num_filters_total = 256*len(filter_sizes) 
            self.h_char_pool = tf.concat(pooled_char_x, 3)
            self.char_x_flat = tf.reshape(self.h_char_pool, [-1, num_filters_total], name="pooled_char_x")
            self.char_h_drop = tf.nn.dropout(self.char_x_flat, self.dropout_keep_prob, name="dropout_char_x")
        
        ############################### CONCAT WORD AND CHAR BRANCH ############################
        if mode == 3 or mode == 5: 
            with tf.name_scope("word_char_concat"): 
                ww = tf.get_variable("ww", shape=(num_filters_total, 512), initializer=tf.contrib.layers.xavier_initializer())
                bw = tf.Variable(tf.constant(0.1, shape=[512]), name="bw") 
                l2_loss += tf.nn.l2_loss(ww) 
                l2_loss += tf.nn.l2_loss(bw) 
                word_output = tf.nn.xw_plus_b(self.h_drop, ww, bw)

                wc = tf.get_variable("wc", shape=(num_filters_total, 512), initializer=tf.contrib.layers.xavier_initializer())
                bc = tf.Variable(tf.constant(0.1, shape=[512]), name="bc") 
                l2_loss += tf.nn.l2_loss(wc)
                l2_loss += tf.nn.l2_loss(bc)
                char_output = tf.nn.xw_plus_b(self.char_h_drop, wc, bc) 
            
                self.conv_output = tf.concat([word_output, char_output], 1)              
        elif mode == 2 or mode == 4: 
            self.conv_output = self.h_drop 
        elif mode == 1: 
            self.conv_output = self.char_h_drop        

        ################################ RELU AND FC ###################################
        with tf.name_scope("output"): 
            w0 = tf.get_variable("w0", shape=[1024, 512], initializer=tf.contrib.layers.xavier_initializer())
            b0 = tf.Variable(tf.constant(0.1, shape=[512]), name="b0") 
            l2_loss += tf.nn.l2_loss(w0) 
            l2_loss += tf.nn.l2_loss(b0) 
            output0 = tf.nn.relu(tf.matmul(self.conv_output, w0) + b0)
            
            w1 = tf.get_variable("w1", shape=[512, 256], initializer=tf.contrib.layers.xavier_initializer()) 
            b1 = tf.Variable(tf.constant(0.1, shape=[256]), name="b1") 
            l2_loss += tf.nn.l2_loss(w1) 
            l2_loss += tf.nn.l2_loss(b1) 
            output1 = tf.nn.relu(tf.matmul(output0, w1) + b1)
            
            w2 = tf.get_variable("w2", shape=[256,128], initializer=tf.contrib.layers.xavier_initializer())
            b2 = tf.Variable(tf.constant(0.1, shape=[128]), name="b2") 
            l2_loss += tf.nn.l2_loss(w2) 
            l2_loss += tf.nn.l2_loss(b2) 
            output2 = tf.nn.relu(tf.matmul(output1, w2) + b2) 
            
            w = tf.get_variable("w", shape=(128, 2), initializer=tf.contrib.layers.xavier_initializer()) 
            b = tf.Variable(tf.constant(0.1, shape=[2]), name="b") 
            l2_loss += tf.nn.l2_loss(w) 
            l2_loss += tf.nn.l2_loss(b) 
            
            self.scores = tf.nn.xw_plus_b(output2, w, b, name="scores") 
            self.predictions = tf.argmax(self.scores, 1, name="predictions") 

        with tf.name_scope("loss"): 
            losses = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores, labels=self.input_y) 
            self.loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss

        with tf.name_scope("accuracy"): 
            correct_preds = tf.equal(self.predictions, tf.argmax(self.input_y, 1)) 
            self.accuracy = tf.reduce_mean(tf.cast(correct_preds, "float"), name="accuracy") 


In [ ]:
# URL들을 그냥 character level로 해서, id로 변환한 것 
chared_id_x = char_id_x(urls, chars_dict, FLAGS["data.max_len_chars"])

# character의 id로 변환한 걸 idx에 매칭시켜서 얻어낸 결과 
x_train_char_seq = get_ngramed_id_x(x_train, chared_id_x)


def batch_iter(data, batch_size, num_epochs, shuffle=True): 
    data = np.array(data) 
    data_size = len(data) 
    
    # 1 epoch 당 batch의 갯수 (1batch당 128개씩 한다고 했으니깐, 그런 batch의 갯수는?) 
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1 
    for epoch in range(num_epochs): 
        if shuffle: 
            shuffle_indices = np.random.permutation(np.arange(data_size)) 
            shuffled_data = data[shuffle_indices]
        else: 
            shuffled_data = data 
        for batch_num in range(num_batches_per_epoch): 
            start_idx = batch_num * batch_size 
            end_idx = min((batch_num+1) * batch_size, data_size)
            
            # 현재 섞여 있는 전체데이터에서, batch의 시작 idx, batch의 끝 idx를 찾아서 그만큼씩을 호출하는 것임. 
            yield shuffled_data[start_idx:end_idx]


'''
x_train_char  : URL의 각각의 단어가 ngram되어서, ngram id로 mapping되어서 나온형태 (char랑 별 다를건 없음)
x_train_word  : URL의 각 단어를 그냥 단어자체의 id로 mapping 시켜서 나온 형태 
x_train_char_seq : URL을 단어로 쪼개는 것도 아니고, 그냥 글자 단위 하나하나로 봐서 character에 대응되는 id로 mapping 시킨 것
'''
def make_batches(x_train_char_seq, x_train_word, x_train_char, y_train, batch_size, nb_epochs, shuffle=False):
    if FLAGS["model.emb_mode"] == 1:  
        batch_data = list(zip(x_train_char_seq, y_train))
    elif FLAGS["model.emb_mode"] == 2:  
        batch_data = list(zip(x_train_word, y_train))
    elif FLAGS["model.emb_mode"] == 3:  
        batch_data = list(zip(x_train_char_seq, x_train_word, y_train))
    elif FLAGS["model.emb_mode"] == 4:
         batch_data = list(zip(x_train_char, x_train_word, y_train))
    elif FLAGS["model.emb_mode"] == 5:  
        batch_data = list(zip(x_train_char, x_train_word, x_train_char_seq, y_train))
        
    # 말 그대로 batch 별로 뽑아내서 iteration 시켜주는 것이다. 
    # 여기에 batch는 하나의 batch이다. 
    batches = batch_iter(batch_data, batch_size, nb_epochs, shuffle)

    if nb_epochs > 1: 
        nb_batches_per_epoch = int(len(batch_data)/batch_size)
        
        # nb_batches_per_epoch 는 그 해당 epoch에서 몇번째 batch 인지를 나타낸다. 
        # 총 몇번째 batch인지 (epoch 통틀어서) 를 nb_batch로 표현한다. 
        if len(batch_data)%batch_size != 0:
            nb_batches_per_epoch += 1
        nb_batches = int(nb_batches_per_epoch * nb_epochs)
        return batches, nb_batches_per_epoch, nb_batches
    else:
        return batches 
    
    
train_batches, nb_batches_per_epoch, nb_batches = make_batches(x_train_char_seq, x_train_word, x_train_char, y_train, 128, 5, True)
    

In [ ]:
'''
x 는 batch인데 mode에 따라 다음과 같이 구성되어 있다. 

pad_idx 가 char-level로 word를 나타내는 것이다. ngram해서. 

mode = 1 [x_char_seq]
mode = 2 [x_word ]
mode = 3 [x_char_seq, x_word]
mode = 4 [x_word, x_char, x_char_pad_idx]
mode = 5 [x_char_seq, x_word, x_char, x_char_pad_idx]
'''

def train_dev_step(x, y, emb_mode, is_train=True):
    if is_train: 
        p = 0.5
    else: 
        p = 1.0
    if emb_mode == 1: 
        feed_dict = {
            cnn.input_x_char_seq: x[0],
            cnn.input_y: y,
            cnn.dropout_keep_prob: p}  
    elif emb_mode == 2: 
        feed_dict = {
            cnn.input_x_word: x[0],
            cnn.input_y: y,
            cnn.dropout_keep_prob: p}
    elif emb_mode == 3: 
        feed_dict = {
            cnn.input_x_char_seq: x[0],
            cnn.input_x_word: x[1],
            cnn.input_y: y,
            cnn.dropout_keep_prob: p}
    elif emb_mode == 4: 
        feed_dict = {
            cnn.input_x_word: x[0],
            cnn.input_x_char: x[1],
            cnn.input_x_char_pad_idx: x[2],
            cnn.input_y: y,
            cnn.dropout_keep_prob: p}
    elif emb_mode == 5:  
        feed_dict = {
            cnn.input_x_char_seq: x[0],
            cnn.input_x_word: x[1],
            cnn.input_x_char: x[2],
            cnn.input_x_char_pad_idx: x[3],
            cnn.input_y: y,
            cnn.dropout_keep_prob: p}
    if is_train:
        _, step, loss, acc = sess.run([train_op, global_step, cnn.loss, cnn.accuracy], feed_dict)
    else: 
        step, loss, acc = sess.run([global_step, cnn.loss, cnn.accuracy], feed_dict)
    return step, loss, acc


# 실제로 학습이 진행되는 부분인듯. 

for idx in it:
    batch = next(train_batches)  # train_batches가 generator, iterator이므로 그때그때마다 next()로해서 batch를 구해온다. 
    x_batch, y_batch = prep_batches(batch) 
    step, loss, acc = train_dev_step(x_batch, y_batch, emb_mode=FLAGS["model.emb_mode"], is_train=True)

In [ ]:
# 주어진 batch이다.  
'''
mode = 1 : only character-based CNN, 
mode = 2 : only word-based CNN, 
mode = 3 : character and word CNN, 
mode = 4 : character-level word CNN, 
mode = 5 : character and character-level word 


urls 는 x_char_seq, 혹은 x_word가 들어가는데, 이건 url을 character별로 쪼개서, character에 대응하는 숫자 id로만 mapping 킨것
max_d1 은 한 URL에 들어갈 수 있는 최대 character 갯수 
'''
def pad_seq_in_word(urls, max_d1=0, embedding_size=128):
    if max_d1 == 0: 
        url_lens = [len(url) for url in urls]
        max_d1 = max(url_lens)
        
    # 일단 최대길이를 기준으로 해서 0으로 다 채운다. [[0,0,0,0]. [0,0,0,0], [0,0,0,0] ] 이런식으로..     
    pad_urls = np.zeros((len(urls), max_d1))
    #pad_idx = np.zeros((len(urls), max_d1, embedding_size))
    #pad_vec = [1 for i in range(embedding_size)]
    
    # 일단 0으로 padding 된 자리에다가, 
    for d0 in range(len(urls)): 
        url = urls[d0]
        for d1 in range(len(url)): 
            if d1 < max_d1: 
                pad_urls[d0,d1] = url[d1]
                #pad_idx[d0,d1] = pad_vec 
    return pad_urls 

'''
pad_seq가 사용되는 경우는 emb_mode가 4,5일 때이다. char-level word인 경우 
urls에 x_char가 들어가는데... x_chars 는 URL의 단어별로 쪼개고, 단어를 ngram단위로 쪼개고 난 다음에, id로 mapping 시킨것. 

이 함수가 하고자 하는 바는, word 단위에서 다시 ngram으로 쪼개서, 그 ngram string이랑, 해당 vector를 적는것 
max_d1 = .max_len_words => 한 URL에 최대로 들어갈 수 있는 단어의 갯수 (200) 
max_d2 = max_len_subwords => 한 단어에 최대로 들어갈 수 있는 단어의 갯수 (20)
'''
def pad_seq(urls, max_d1=0, max_d2=0, embedding_size=128): 
    
    # 정해져있지 않은경우, 찾는과정. 
    if max_d1 == 0 and max_d2 == 0: 
        for url in urls: 
            if len(url) > max_d1: 
                max_d1 = len(url) 
            for word in url: 
                if len(word) > max_d2: 
                    max_d2 = len(word) 
                    
    # pad_idx 는 URL 내 단어의 character들을 받는다. 근데 그 character는 embedding size의 vec로 표현된다.
    # 따라서 pad_idx의 차원은 URL 갯수 * URL내 최대 단어갯수 * 단어 내 최대 글자갯수 * 글자의 embedding vector 수 
    pad_idx = np.zeros((len(urls), max_d1, max_d2, embedding_size))
    pad_urls = np.zeros((len(urls), max_d1, max_d2))
    
    # 해당 character가 존재함을 표시하기 위해서 [1,1,1,1,1] 과 같은 vector로 표현한다. 
    pad_vec = [1 for i in range(embedding_size)]
    for d0 in range(len(urls)): 
        url = urls[d0]
        for d1 in range(len(url)): 
            
            if d1 < max_d1: 
                # word는 ngram으로 이루어진 단어 => emb_mode가 4,5 일때.. 
                word = url[d1]
                
                # 한 단어 내에서 character 나, ngram에 접근할 때는. 한 단어가 최대 몇개의 character로 이뤄지는지를 알아야함.
                for d2 in range(len(word)): 
                    if d2 < max_d2: 
                        # 한 단어의 character를 넣는다. . 
                        pad_urls[d0,d1,d2] = word[d2]  # 그 string으로 들어간다. 
                        
                        # pad
                        pad_idx[d0,d1,d2] = pad_vec  # 해당 string이 embedding vector로 들어간다. 
                        
    # pad_urls 는 단                
    return pad_urls, pad_idx



'''
x_char_seq 는 URL의 character를 숫자 id로 매칭시켜서 얻어낸 결과임 

'''
def prep_batches(batch):
    
    # batch 는 다음과 같이 구성되어 있따. x_train_char, x_train_word, x_train_char_seq, y_train
    if FLAGS["model.emb_mode"] == 1:
        x_char_seq, y_batch = zip(*batch)
    elif FLAGS["model.emb_mode"] == 2:
        x_word, y_batch = zip(*batch)
    elif FLAGS["model.emb_mode"] == 3:
        x_char_seq, x_word, y_batch = zip(*batch)
    elif FLAGS["model.emb_mode"] == 4:
        x_char, x_word, y_batch = zip(*batch)
    elif FLAGS["model.emb_mode"] == 5:
        x_char, x_word, x_char_seq, y_batch = zip(*batch)

    x_batch = []
    
    # batch를 통해서 얻어낸걸, 모델에 투입하기 전에, 그 길이에 맞춰서 
    if FLAGS["model.emb_mode"] in [1, 3, 5]:
        #  URL을 그냥 하나의 char단위로만 쭉 펼쳤을 때도 고려해야 하므로.
        x_char_seq = pad_seq_in_word(x_char_seq, FLAGS["data.max_len_chars"])
        x_batch.append(x_char_seq)
    if FLAGS["model.emb_mode"] in [2, 3, 4, 5]:
        
        x_word = pad_seq_in_word(x_word, FLAGS["data.max_len_words"])
        x_batch.append(x_word)
    if FLAGS["model.emb_mode"] in [4, 5]:
        
        # x_char 는 URL을 다시 word 단위로 쪼개고, word에서 ngram으로 나눠 들어가서 살펴봤을 때이다. 
        # 무튼 x_char는 padding으로 채워진 값이면서, size가[URL 갯수, URL당 word갯수, word당 maximum 글자갯수]
        x_char, x_char_pad_idx = pad_seq(x_char, FLAGS["data.max_len_words"], FLAGS["data.max_len_subwords"], FLAGS["model.emb_dim"])
        
        # emb_mode가 4,5 인 경우에는 아래 경우들도 추가해야한다. 
        x_batch.extend([x_char, x_char_pad_idx])
        
    return x_batch, y_batch